In [ ]:
import numpy as np
import pandas as pd
import fileinput
import json
from tqdm import tqdm
tqdm.pandas()

import pickle

from tssplit import tssplit
import re

!pip3 install numpy

In [ ]:
%%time

df = pd.read_json('/home/dave/NLP/WashingtonPost-v4/data/TREC_Washington_Post_collection.v4.jl', lines=True)
  


ValueError: Expected object or value

In [ ]:
#im ersten durchlauf gedropped
df.drop('article_url', inplace=True, axis=1)
df.drop('published_date', inplace=True, axis=1)
df.drop('content', inplace=True, axis=1)
df.drop('orig-id', inplace=True, axis=1)

In [ ]:
#im zweiten durchlauf gedropped
df.drop('id', inplace=True, axis=1)
df.drop('type', inplace=True, axis=1)
df.drop('source', inplace=True, axis=1)


In [ ]:
df.head()

,title,author,contents
0,"Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Mark Giannotto,"[{'content': 'Colleges', 'mime': 'text/plain',..."
1,Bubbly planet Venus starts off New Year,Blaine P. Friedlander Jr.,"[{'content': 'Local', 'mime': 'text/plain', 't..."
2,Iowa caucuses: GOP presidential hopefuls makin...,Karen Tumulty,"[{'content': 'Politics', 'mime': 'text/plain',..."
3,"John Wall, Wizards seek to pick up their spirits",Michael Lee,"[{'content': 'Wizards/NBA', 'mime': 'text/plai..."
4,Developer of Maryland housing office in Prince...,Ovetta Wiggins,"[{'content': 'Local', 'mime': 'text/plain', 't..."


# 2

In [ ]:
def remove_duplicates(l):
    return list(set(l))

In [ ]:
pattern_auth = "\s*\w*"
pattern_at_empty = "" #(\:*\s*'')
drop_list = []
index = 0
i = 0



for auth in df["author"]:
    at = re.search(pattern_auth, str(auth))
    #at2 = re.search(pattern_at_empty, str(auth))
    if at or len(str(auth)) <= 4:
        drop_list.append(i)
      #print(i)
    if auth == "":
        drop_list.append(i)
    i+= 1
    
### neu
for tit in df["title"]:
  i = 0
  if len(str(tit)) <= 4:
    drop_list.append(i)
  if auth == "":
    drop_list.append(i)
  i+= 1
  


In [ ]:
df = df.drop(remove_duplicates(drop_list))

In [ ]:

df = df.sort_index()
df = df.reset_index(drop=True)


In [ ]:
%%time
le = int(len(df)/8)
start = 0
end = le
i = 1 
while i < 9:
    df[start:end].to_pickle("/home/dave/NLP/WashingtonPost-v4/data/"+str(i)+".pkl")
    start = end
    end = end + le
    i += 1

CPU times: user 1min 20s, sys: 7.49 s, total: 1min 27s
Wall time: 1min 28s


# Kernel neu starten

In [ ]:
import numpy as np
import pandas as pd
import fileinput
import json
from tqdm import tqdm
tqdm.pandas()

import pickle

from tssplit import tssplit
import re

In [ ]:
%%time
pattern_paragraph = "\'subtype\': \'paragraph\'"
pattern_date = "\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z"



for c in range(1,9):
    
    df = pd.read_pickle("/home/dave/NLP/WashingtonPost-v4/data/"+str(c)+".pkl")
    
    df = df.reset_index(drop=True)
    
    meta_List = []
    content_List = []

    j= 0

    for article in df["contents"]:
        temp_List_meta = []
        temp_List_content = []
        temp_List_content2 = []
        temp_List_meta2 = []
        temp_List_meta4 = []

        for eintrag in article:
            x = re.search(pattern_paragraph, str(eintrag))
            if x:
                temp_List_content = [eintrag]
                for t in temp_List_content:
                    val = str(t)
                    val3 = tssplit(val,quote="'", delimiter=",")
                    t4 = val3[0][10:]
                    temp_List_content2.append(t4)
            else:
                temp_List_meta = [eintrag]
                for t in temp_List_meta:

                    val2 = str(t)
                    val6 = val2.split(",")
                    t8 = val6[0][13:]
                    temp_List_meta2.append(t8)
        
        
        
                
        temp_List_meta4.append(temp_List_meta2[0])
        temp_List_meta4.append(df["title"][j])   ### neu
        temp_List_meta4.append(df["author"][j])

        


        for m in temp_List_meta2:                
            d = re.search(pattern_date, str(m))          
            if d:
                temp_List_meta4.append(m)
            else:
                continue
        
        
        #bool_kat = ist_in_kat(temp_List_meta2[0])## neu
        #if bool_kat:
        if len(temp_List_meta4) >= 3 and len(temp_List_meta4) <= 4 and len(temp_List_content2) > 3:
            #if temp_List_meta4[0]
            meta_List.append(temp_List_meta4)    
            content_List.append("".join(temp_List_content2))
        j+=1        

    dfMeta = pd.DataFrame(meta_List)
    dfContent = pd.DataFrame(content_List)

    dfContent = dfContent.set_axis(["content"], axis=1, inplace = False)

    dfMeta = dfMeta.set_axis(["Kategorie","Titel","Autor","Datum"], axis=1, inplace = False)

    dfContent["id"] = dfContent.index
    dfMeta["id1"] = dfMeta.index

    df = pd.merge(dfMeta,dfContent, left_on="id1", right_on="id", how="left").drop("id1",axis=1)
    
    df.to_pickle("/home/dave/NLP/WashingtonPost-v4/data/"+str(c)+"_raw.pkl")

    c += 1

CPU times: user 17min 6s, sys: 21.1 s, total: 17min 27s
Wall time: 17min 27s


# Kernel neu starten

## 5

In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
tqdm.pandas()
import fileinput
import pickle
import re
import spacy
from pandarallel import pandarallel

!python -m spacy download en_core_web_md <br>
!python -m spacy download en_core_web_lg

In [ ]:
%%time

for c in range(1,9):
    
    df = pd.read_pickle("/home/dave/NLP/WashingtonPost-v4/data/"+str(c)+"_raw.pkl")
    
    df = df.set_index('id')
    
    df.Kategorie = df.Kategorie.replace("\"$|\'$", "",regex=True)
    df.Titel = df.Titel.replace("\"$|\'$", "",regex=True)
    df.Datum = df.Datum.replace("\"$|\'$", "",regex=True) 

    df.Titel = df.Titel.replace("\<.*?\>","",regex=True)
    df.content = df.content.replace("\<.*?\>","",regex=True)
    
    df.to_pickle("data/"+str(c)+"_clean.pkl")

CPU times: user 38.4 s, sys: 5.88 s, total: 44.3 s
Wall time: 44.3 s


# Kernel neu starten

In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
tqdm.pandas()
import fileinput
import pickle
import re

In [ ]:
dfKat_behalten = pd.read_csv("/home/dave/NLP/WashingtonPost-v4/data/Kategorie_behalten.CSV",header=None)

Kat_behalten = list(dfKat_behalten[0])


In [ ]:
%%time
temp_list = []

for c in range(1,9):
    
    df = pd.read_pickle("/home/dave/NLP/WashingtonPost-v4/data/"+str(c)+"_clean.pkl")
    i=0
    dfEnd = pd.DataFrame()
    for a in df.iterrows():
        if a[1][0] in Kat_behalten:
            #print("isda")
            temp_list.append(a[1])
        i+= 1
    
dfEnd = pd.DataFrame(temp_list)

CPU times: user 43.5 s, sys: 6.07 s, total: 49.6 s
Wall time: 49.6 s


In [ ]:
dfEnd = dfEnd.reset_index()
dfEnd = dfEnd.drop("index", axis=1)
dfEnd = dfEnd.reset_index()
dfEnd = dfEnd.set_index('index')  

In [ ]:
%%time
anzahl = 4

le = int(len(dfEnd)/anzahl)
start = 0
end = le
i = 1 
while i < (anzahl+1):
    dfEnd[start:end].to_pickle("/home/dave/NLP/WashingtonPost-v4/data/rdy/"+str(anzahl)+"/"+str(i)+"_rdy.pkl")
    start = end
    end = end + le
    i += 1

CPU times: user 624 ms, sys: 2.04 s, total: 2.66 s
Wall time: 10.4 s


In [ ]:
dfEnd.to_pickle("data/rdy/all_rdy.pkl")

# Kernel neu starten

In [ ]:
#!pip3 install torch


Usage:   
  pip3 uninstall [options] <package> ...
  pip3 uninstall [options] -r <requirements file> ...

no such option: -Y


In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
tqdm.pandas()
import fileinput
import pickle
import re
import spacy
import glob
from pandarallel import pandarallel

##cuda
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
spacy.prefer_gpu() 

False

In [2]:
nlp = spacy.load("en_core_web_lg")
pandarallel.initialize(progress_bar=False)

INFO: Pandarallel will run on 22 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
all_stopwords = nlp.Defaults.stop_words
def remove_stopwords_from_list(list_in):
    return [a for a in list_in if a not in all_stopwords]

In [4]:
filename=0
for files in glob.glob("/home/dave/NLP/WashingtonPost-v4/data/rdy/16/*.pkl"):
    print("reading from: "+ files)
    df = pd.read_pickle(files)
    filename+=1
    df["tokens"] = df.parallel_apply(lambda row: [a.text for a in list(nlp(str(row["content"])))], axis=1)
    df["no_stop"] = df.parallel_apply(lambda row: remove_stopwords_from_list(row["tokens"]), axis=1)
    df["lemma"] = df.parallel_apply(lambda row: [t.lemma_ for t in nlp(str(row["no_stop"]))], axis=1)
    df.to_pickle("/home/dave/NLP/WashingtonPost-v4/data/nlp/{}_nlp.pkl".format(filename))
    print("saved file as {}_nlp.pkl".format(filename))
    


reading from: /home/dave/NLP/WashingtonPost-v4/data/rdy/16/8_rdy.pkl
saved file as 1_nlp.pkl
reading from: /home/dave/NLP/WashingtonPost-v4/data/rdy/16/11_rdy.pkl
saved file as 2_nlp.pkl
reading from: /home/dave/NLP/WashingtonPost-v4/data/rdy/16/2_rdy.pkl


KeyboardInterrupt: 

In [ ]:
df

,Kategorie,Titel,Autor,Datum,content,tokens,no_stop
index,,,,,,,
28712,Maryland Politics,Montgomery County asks U.S. to reopen shuttere...,Bill Turque,2013-10-14T22:42:55Z,Montgomery County has asked the U.S. Departmen...,"[Montgomery, County, has, asked, the, U.S., De...","[Montgomery, County, asked, U.S., Department, ..."
28713,Local,How many birds die from hitting D.C. buildings...,John Kelly,2013-10-14T22:48:00Z,The birds are still warm to the touch when Nic...,"[The, birds, are, still, warm, to, the, touch,...","[The, birds, warm, touch, Nick, Lund, lifts, c..."
28714,The Fed Page,Government shutdown generates stories of miser...,Joe Davidson,2013-10-14T22:49:21Z,As Congress and the White House try to find th...,"[As, Congress, and, the, White, House, try, to...","[As, Congress, White, House, try, find, way, b..."
28715,National Security,NSA collects millions of e-mail address books ...,Barton Gellman; Ashkan Soltani,2013-10-14T22:53:00Z,The National Security Agency is harvesting hun...,"[The, National, Security, Agency, is, harvesti...","[The, National, Security, Agency, harvesting, ..."
28716,Politics,Self-cast group of Senate moderates wants to m...,David A. Fahrenthold; and Rosalind S. Helderman,2013-10-14T23:23:39Z,"In the past few days, a bipartisan group of 12...","[In, the, past, few, days, ,, a, bipartisan, g...","[In, past, days, ,, bipartisan, group, 12, sen..."
...,...,...,...,...,...,...,...
43063,Capital Business,Editor’s Note: The key to success? Have more c...,Dan Beyers,2014-09-07T19:29:22Z,"Recently, I found myself in the halls of the B...","[Recently, ,, I, found, myself, in, the, halls...","[Recently, ,, I, found, halls, Bethesda, North..."
43064,Capital Business,Capital Buzz: Entrepreneurial brothers create ...,Thomas Heath,2014-09-07T19:30:45Z,Two Washington entrepreneurs with a history of...,"[Two, Washington, entrepreneurs, with, a, hist...","[Two, Washington, entrepreneurs, history, buil..."
43065,Capital Business,"CoStar: Retail activity is strong in D.C., but...",Kirstie Boatright,2014-09-07T19:31:32Z,Although popular retailers such as Kate Spade ...,"[Although, popular, retailers, such, as, Kate,...","[Although, popular, retailers, Kate, Spade, Tr..."


In [ ]:
df = pd.read_pickle("/home/dave/NLP/WashingtonPost-v4/data/rdy/16/2_basic_NLP.pkl")